# Year 2023 Day 11


The goal of this notebook is multiple:

1. Learn how to use Observable Plot with its python wrapper `pyobsplot`
2. Visualize the problem's input
3. Describe the problem solving procedure


In [ ]:
from advent_of_code.visualization.observable_plot import visualize_puzzle_input_202311
from advent_of_code.y_2023.problem_202311 import (
    AdventOfCodeProblem202311,
    create_coord_array,
)

### Input Visualization


In [ ]:
problem = AdventOfCodeProblem202311()
problem

In [ ]:
puzzle_input = problem.parse_input_text_file()
puzzle_input

In [ ]:
visualize_puzzle_input_202311(puzzle_input)

In [ ]:
visualize_puzzle_input_202311(puzzle_input, with_rules=True)

Let's count how many nodes there are in this graph:


In [ ]:
coord_array = create_coord_array(puzzle_input)
node_count = coord_array["z"].size
node_count

Now we can calculate the number of edges in the related [Complete graph](https://en.wikipedia.org/wiki/Complete_graph), where every pair of node is connected:


In [ ]:
n = node_count
edge_count = n * (n - 1) // 2  # 2 among n
edge_count

The amount is very high, hence in the visualisation, only a fraction is shown


In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.005,
)

In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.03,
)

We can visually see that there are a lot of distances to compute. The plot is filled with edges. Here is an edge of density of 0.1, we can see that the plot is almost filled with the drawing color:


In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.1,
)

The key to expansion is to assign to every chunk a node (it creates a [Lattice graph](https://en.wikipedia.org/wiki/Lattice_graph)).
Then the same logic can be applied on chunks instead of stars, and the distance
becomes the sum of both adjacency matrix, with the chunks adjacency matrix multiplied by
the expansion coefficient


In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_chunk_graph=True,
)

## Problem Solving


In [ ]:
from advent_of_code.y_2023.problem_202311 import (
    compute_proximity_matrix_from_coord_array,
    create_chunk_coord_array,
    get_expansion_coef_part_2,
)
import pandas as pd

First, let's visualize again the puzzle input, an boolean array containing the stars.


In [ ]:
space_xda = puzzle_input
visualize_puzzle_input_202311(puzzle_input)

Then, create an array containing all the coordinates of the stars in the puzzle input


In [ ]:
coord_array = create_coord_array(space_xda)
coord_array.to_pandas()

Each star is contained in a chunk. Similarly, create an array containing all the _chunks_ coordinates.


In [ ]:
chunk_coord_array = create_chunk_coord_array(space_xda, coord_array)
chunk_coord_array.to_pandas()

Now, let's compute the _proximity matrix_ for stars, $P_{stars}$


In [ ]:
proximity_matrix_stars = compute_proximity_matrix_from_coord_array(coord_array)
proximity_matrix_stars.to_pandas()

Then, compute the _proximity matrix_ for chunks, $P_{chunks}$


In [ ]:
proximity_matrix_chunks = compute_proximity_matrix_from_coord_array(chunk_coord_array)
proximity_matrix_chunks.to_pandas()

The expansion coefficient $e_{coef}$ is given by the problem description


In [ ]:
expansion_coef = get_expansion_coef_part_2()
expansion_coef

The _total proximity matrix_ can then be computed with this expression:

$$
P_{total} = P_{stars} + e_{coef} .  P_{chunks}
$$


In [ ]:
total_proximity = proximity_matrix_stars + expansion_coef * proximity_matrix_chunks
total_proximity.to_pandas()

Finally, the answer to the problem can be obtained by summing either the lower or upper triangular matrix extracted from $P_{total}$.
